In [12]:
# 05/08/2025 
# Indices updated, not tested

using Random, Distributions, Optim, NLsolve, SpecialFunctions
N = 2 # Number of countries
periods = 5
J = 3 # Number of goods (needs to be big number and an integer type)
θ = fill(4.0, J) # Frechet shape parameter (governs comparative advantage)

v = 1.0 # migration elasticity
β = .99 # discount factor
α = ones(J) * (1/J) # final good expenditure share

τ = ones(N, N, J) # Iceberg trade costs 
Lt = ones(N, J, periods) # Size of labor force in each country at time 

Ldot = ones(N, J, periods)

At0 = ones(N, J)#initial productivities


wt = ones(N, J, periods)
wt0 = ones(N, J)

wdot = ones(N, J, periods)
tradesharest0 = ones(N, N, J) * (1 / N)
Adot = ones(N, J, periods)


kdot = ones(N, N, J, periods)

pdotArray = ones(N, J, periods)
d1wdot = ones(N, J)

μtminus1 = zeros(N, N, J, J) #value of μ at time t = -1
πt0 = ones(N, N, J) #initial trade shares
μt = ones(N, N, J, J, periods)*(1/(N*J))

# a guess for path of udot:
udotPathGuess = ones(N, J, periods)
udotPathUpdate = zeros(N, J, periods)
errormax = 1.0 #the maximum log difference between guesses and updates for appendix D algorithm


function pdot(n, j, d1wdot, kdot, Adot, time, tradesharest0) #pdot(nj) from equation (12)
    (sum(tradesharest0[n, i, j] * (d1wdot[i, j] * kdot[n, i, j, time])^-θ[j] * Adot[i, j, time]^θ[j] for i in 1:N))^(-1 / θ[j])
end 

function tradeSharest0(n, i, j, wt0, At0, τ) #trade shares to nj from ij, equation (7)
    wt0[i, j] * τ[n, i, j]^-θ[j] * At0[i, j] ^θ[j] / (sum((wt0[m, j] *τ[n, m, j])^-θ[j] * At0[m, j]^θ[j] for m in 1:N)) 
end

function tradeSharest1(n, i, j, d1wdot, Ldot, Adot, kdot, time, tradesharest0) #trade shares to nj from ij, equation (13)
    tradesharest0[n, i, j] * ((d1wdot[i, j] * kdot[n, i, j, time]) / pdot(n, j, d1wdot, kdot, Adot, time, tradesharest0))^-θ[j] * Adot[i, j, time]^θ[j]
end

function incomet0(n, j, wt0, Lt) # total income of country n, sector j in time t=0 given wage and labor  
    wt0[n, j] * Lt[n, j, 1] 
end

function incomet1(n, j, d1wdot, time, Lt, wt) # total income of country n, sector j in time t+1
    wt[n, j, time]*(d1wdot[n, j])*Lt[n, j, time]*(Ldot[n, j, time])
end

function Xt0(n, j, α, wt0, Lt) # expenditure on sector good j in region n, from equation (8)
    α[j] * sum(wt0[n, k] * Lt[n, k, 1] for k in 1:J) 
end

function Xt1(n, j, α, d1wdot, Lt, wt, Ldot, time) # expenditure on sector good j in region n, from equation (14)
    α[j] * sum(d1wdot[n, k] * Ldot[n, k, time] * wt[n, k, time] * Lt[n, k, time] for k in 1:J) #from equation (14) 
end


while errormax > .01 
    for time in 1:periods-1
        # Update μt step 2 of appendix D
        for n in 1:N               # destination country
            for i in 1:N           # source country
                for j in 1:J       # destination sector
                    for k in 1:J   # source sector
                        num = μt[n, i, j, k, time] *
                            (udotPathGuess[i, k, time+1])^(β/v)

                        # denominator: sum over all (m, h)
                        denom = sum(μt[n, m, j, h, time] *(udotPathGuess[m, h, time+1])^(β/v) for m in 1:N, h in 1:J)
                        μt[n, i, j, k, time+1] = num / denom
                    end
                end
            end
        end

        # Update Lt step 3 of appendix D
        for n in 1:N
            for j in 1:J
                Lt[n, j, time+1] = sum(μt[i, n, k, j, time] * Lt[i, k, time] for i in 1:N, k in 1:J)
            end
        end
        # Update Ldots from Lts
        Ldot[:, :, time] .= Lt[:, :, time+1] ./ Lt[:, :, time]
    end

    function g!(G, wt0)
        for n in 1:N
            for j in 1:J
                G[n, j] = incomet0(n, j, wt0, Lt) - sum(Xt0(i, j, α, wt0, Lt) * tradeSharest0(n, i, j, wt0, At0, τ) for i in 1:N) #market clearing for wt0
            end
        end
    end
    initial = [1.0  1.1  1.2;
           1.15 1.22 1.21]    # size: 2×3 if N=2, J=3

    results = nlsolve(g!, initial) #solve for wages

    # extract the wage solution as an N×J matrix
    wt0 = results.zero
    
    wt[:,:, 1] .= wt0[:,:]

    for n in 1:N
        for i in 1:N
            for j in 1:J
                tradesharest0[n, i , j] = tradeSharest0(n, i, j, wt0, At0, τ)
            end
        end
    end

    println(wt0)  #end solving for wt0

    for time in 1:periods-1
        function f!(F, d1wdot)
            for n in 1:N
                for j in 1:J
                    F[n,j] = (incomet1(n, j, d1wdot, time, Lt, wt) - sum(tradeSharest1(n, i, j, d1wdot, Ldot, Adot, kdot, time, tradesharest0)
                    * Xt1(i, j, α, d1wdot, Lt, wt, Ldot, time) for i in 1:N)) # equation (15)
                end
            end
        end
        println("checkpoint")
        #initial = [1.0  1.1  1.2;
        #        1.15 1.22 1.21]    # size: 2×3 if N=2, J=3
        #results = nlsolve(f!, initial) #solving for wages with country 1 set at 1.0
        initial_rand = rand(N,J)
        nlsolve(f!, initial_rand)
        println("Random guesses:", initial_rand)
    
        wdot[:, :, time] .= results.zero[:,:] # updates wdot with solution to the system of equations
        d1wdot[:,:] .= wdot[:,:,time] 

        for n in 1:N, j in 1:J
            println(wdot[n, j, time])
        end

        println("checkpoint")
        ################### not edited past this point
        for n in 1:N, j in 1:J
            pdotArray[n,j,time] = pdot(n, j, d1wdot, kdot, Adot,time, tradesharest0)
        end
    
        tradesharest0[:,:,:] .= [tradeSharest1(n,i,j,d1wdot,Ldot,Adot,kdot,time, tradesharest0) for n in 1:N, i in 1:N, j in 1:J]

        wt[:, :, time + 1] .= wdot[:, :, time] .* wt[:, :, time]

    end

    for time in 1:periods-1
        for n in 1:N, j in 1:J
            udotPathUpdate[n,j,time] = wdot[n,j,time]*(sum(μt[n,i,j,k,time]*(udotPathGuess[i,k,time+1])^(β/v) for i in 1:N, k in 1:J))^(v) ##needs verification but is equation 17 as specified by step 5
        end## Note: trying to figure out how to make sure the time+1 does not end the program with infs or NaN in the error
    end
    # Take the log difference of the guess and updated udots to get the error
    logudotPathGuess = log.(udotPathGuess) 

    logudotPathUpdate = log.(udotPathUpdate)
    logdifference = abs.(logudotPathGuess - logudotPathUpdate)
    errormax = maximum(logdifference[:,:,1:5])

    udotPathGuess = udotPathUpdate
    println("WHILE LOOP")
    display(errormax)

end


[0.9999999999999322 0.9999999999999325 1.0000000000008966; 0.9999999999999322 0.9999999999999325 1.0000000000008966]
checkpoint
Random guesses:[0.3770656147066499 0.3618410307332891 0.7501879177731171; 0.46809487296691266 0.3601377248344467 0.16584404247347395]
0.9999999999999322
0.9999999999999325
1.0000000000008966
0.9999999999999322
0.9999999999999325
1.0000000000008966
checkpoint
checkpoint
Random guesses:[0.03221993327626427 0.6914885908095428 0.4199442275531613; 0.24078671692634168 0.5043687325618846 0.08823386659581434]
0.9999999999999322
0.9999999999999325
1.0000000000008966
0.9999999999999322
0.9999999999999325
1.0000000000008966
checkpoint
checkpoint
Random guesses:[0.29511303640699416 0.3756383316866372 0.16584841897102165; 0.8333683643419793 0.9685328790157344 0.7255002064588874]
0.9999999999999322
0.9999999999999325
1.0000000000008966
0.9999999999999322
0.9999999999999325
1.0000000000008966
checkpoint
checkpoint
Random guesses:[0.9801594311681681 0.6451415848135408 0.08355

Inf

NaN

In [24]:
#2/28/2025 with d1wdot instead of wdot
using Random, Distributions, Optim, NLsolve, SpecialFunctions
N = 2 # Number of countries
periods = 7
J = 3 # Number of goods (needs to be big number and an integer type) 
θ = fill(4.0, J) # Frechet shape parameter (governs comparative advantage)
T = ones(N*J, periods) # Frechet scale parameter (governs absolute advantage)
σ = 2 # Substitution elasticity between goods
τ = ones(N*J, N, periods) # Iceberg trade costs
L = ones(N*J, periods) # Size of labor force in each country
Ldot = ones(N*J, periods)


wt = ones(N*J, periods)
Lt = ones(N*J, periods)
wdot = ones(N*J, periods)
tradesharest0 = ones(N*J, N) * (1 / N)
Adot = ones(N*J, periods)

kdot = ones(N*J, N, periods)
TSt1 = ones(N*J, N, periods) 
pdotArray = ones(N*J, periods)
d1wdot = ones(N*J)
μ = zeros(N*J, N*J, periods)

μtminus1 = ones(N*J, N*J) #value of μ at time t = -1
πt0 = ones(N*J, N) #initial trade shares
wLt0 = ones(N*J, N) #initial values of w*L
rHt0 = ones(N*J, N) #initial values of r*H
μt = zeros(N*J,N*J,periods)

# a guess for path of udot:
udotPathGuess = ones(N*J, periods)
udotPathUpdate = zeros(N*J, periods)
errormax = 1.0 #the maximum log difference between guesses and updates for appendix D algorithm

function θindex(j)
    div(j-1, N) + 1  # Find the correct index for θ
end

function pdot(n, d1wdot, kdot, Adot, time)
    j = θindex(n)
    (sum(tradesharest0[n, i] * (d1wdot[(j-1) * N + i] * kdot[n, i, time])^-θ[j] * Adot[(j-1) * N + i, time]^θ[j] for i in 1:N))^(-1 / θ[j])
end ##### Going to need to update tradeshares

function tradeSharest1(n, i, d1wdot, Ldot, Adot, kdot, time)
    j = θindex(n)
    tradesharest0[n, i] * ((d1wdot[(j-1) * N + i] * kdot[n, i, time]) / pdot(n, d1wdot, kdot, Adot, time))^-θ[j] * Adot[(j-1) * N + i, time]^θ[j]
end
function incomet1(n, d1wdot, time)
    wt[n, time]*(d1wdot[n])*L[n, time]*(Ldot[n, time])
end
function Xt1(n, α, d1wdot, tradesharest0, Ldot, Adot, kdot, time)
    sum(incomet1(n, d1wdot, time) * tradeSharest1(n, i, d1wdot, Ldot, Adot, kdot, time) for i in 1:N)
end



while errormax > .01 
    for time in 1:periods-1
        for n in 1:N*J
            for i in 1:N*J
                if time == 1
                    μt[n,i,time] = μt[n,i,time] + (μtminus1[n,i]*udotPathGuess[i,time+1])/sum(μtminus1[n,i]*udotPathGuess[h,time+1] for h in 1:N*J)
                else 
                    μt[n,i,time] = μt[n,i,time] + (μt[n,i,time-1]*udotPathGuess[i,time+1])/sum(μt[n,h,time-1]*udotPathGuess[h,time+1] for h in 1:N*J)
                end
            end
        end
        for i in 1:N*J ### Step 3 of the appendix D
            for j in 1:N*J
                Lt[i ,time + 1] = Lt[i ,time + 1] + Lt[j, time]*μt[i,j,time]
            end
        end
    end

    for time in 1:periods-1
        for n in 1:N*J
            for i in 1:N
                TSt1[n, i, time] = tradeSharest1(n, i, d1wdot, Ldot, Adot, kdot, time)
            end
        end
        #############################################
        function f!(F, d1wdot)
            for n in 1:N*J
                j = div(n-1, N) + 1  # Find the correct index for j
                country = n % N 
                if country == 0
                    country = N
                end
                #F[n] = (d1wdot[n] * Ldot[n,time] * wt[n,time] * Lt[n,time] - sum(tradeSharest1((i-1) * J + j, country, d1wdot, Ldot, Adot, kdot, time)
                #    * Xt1((i-1) * J + j, d1wdot, tradesharest0, Ldot, Adot, kdot, time) for i in 1:N)) # Wrong indexing!!!
                F[n] = (incomet1(n, d1wdot, time) - sum(tradeSharest1((j-1) * N + i, country, d1wdot, Ldot, Adot, kdot, time)
                    * Xt1((j-1) * N + i, d1wdot, tradesharest0, Ldot, Adot, kdot, time) for i in 1:N))
            end
        end
        println("checkpoint")
        results = nlsolve(f!, [1.0; 1.1; 1.2; 1.15; 1.22; 1.21]) #solving for wages with country 1 set at 1.0
        sol = ones(N*J, periods)
        for i = 1:N*J
            sol[i, time] = results.zero[i] #saving results in sol[i]
            wdot[i, time] = sol[i,time]
            println(sol[i,time])
            wdot[i, time] = sol[i, time]
        end
        println("checkpoint")
        for n in 1:N
            pdotArray[n,time] = pdot(n, wdot, kdot, Adot,time)
        end
    
        for n in 1:N*J 
            for i in 1:N
                tradesharest0[n,i] = TSt1[n,i,time]
            end
        end
        for i in 1:N*J
            wt[i, time + 1] = wdot[i,time]*wt[i,time]
        end
    end

    for time in 1:periods-1
        for n in 1:N*J
            udotPathUpdate[n,time] = wt[n,time]*(sum(μt[n,i,time]*udotPathGuess[i,time+1] for i in N*J)) ##needs verification but is equation 17 as specified by step 5
        end ## Note: trying to figure out how to make sure the time+1 does not end the program with infs or NaN in the error
    end
    # Take the log difference of the guess and updated udots to get the error
    logudotPathGuess = log.(udotPathGuess) 

    logudotPathUpdate = log.(udotPathUpdate)
    logdifference = abs.(logudotPathGuess - logudotPathUpdate)
    errormax = maximum(logdifference[:,1:5])

    udotPathGuess = udotPathUpdate
    println("WHILE LOOP")
    display(errormax)

end

checkpoint


BoundsError: BoundsError: attempt to access 6×2×7 Array{Float64, 3} at index [1, 1]

In [ ]:

import Pkg; Pkg.add("SpecialFunctions")

   Resolving package versions...
    Updating `~/.julia/environments/v1.10/Project.toml`
  [276daf66] + SpecialFunctions v2.5.0
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
